In [ ]:
import os
import pandas as pd
from rdkit import Chem

train_path='./source/train'
valid1_path='./source/valid1'
valid2_path='./source/valid2'

In [ ]:
# convert cas-smiles excel files to csv
# excel_path is the path for cas-smiles excel raw files

excel_path=''
print('number of excel files: ',len(os.listdir(excel_path)))


for i in os.listdir(excel_path):
    if '.xlsx' in i:
        read_file = pd.read_excel(excel_path+'/'+i)
        newname=i.split('.')[0]+'.csv'
        read_file.to_csv (excel_path+'/'+newname, index = False,header=True)
        print(newname,read_file.shape[0])

In [ ]:
# convert cas-smiles sdf files to csv
# sdf_path is the path for cas-smiles sdf raw files


sdf_path=''
print('number of sdf files: ',len(os.listdir(sdf_path)))


def split_file(file_path, delimiter):
    with open(file_path, 'r') as file:
        content = file.read()
        lines = content.split(delimiter)
        return lines


for i in os.listdir(sdf_path):
    cas_smiles=pd.DataFrame(columns=['CAS Registry Number','Canonical SMILES'])
    with open(sdf_path+'/'+i, 'r') as file:
        z=0
        # z documents the number of CAS
        for line in file:
            if 'Copyright (C)' in line:
                cas_smiles.loc[z,'CAS Registry Number']=line.split(' ')[0]
                z+=1
    single_sdf = split_file(sdf_path+'/'+i, '$$$$')
    for ii in range(1,len(single_sdf)):
        single_sdf[ii] = single_sdf[ii].removeprefix('\n')
    for ii in range(0,len(single_sdf)-1):
        cas_smiles.loc[ii,'Canonical SMILES']=Chem.MolToSmiles(Chem.MolFromMolBlock(single_sdf[ii],sanitize=False))
    
    cas_smiles.to_csv(sdf_path+'/'+i.split('.')[0]+'.csv',sep=',',index=False)
    
    print(i,z)

In [ ]:
# drop duplicates, drop wrong smiles, confirm key number in each file

key_file=pd.read_excel('valid2.xlsx')

for i in os.listdir(valid2_path):
    df=pd.read_csv(valid2_path+'/'+i)
    df.drop_duplicates()
    smiles_list = df['Canonical SMILES'].tolist()
    
    Mols=[]
    for ii in smiles_list:
        try:
            mol=Chem.MolFromSmiles(ii)
        except:
            mol=None
        Mols.append(mol)
    
    df['Mol']=Mols
    df = df.dropna(how='any',axis=0)
    df = df.drop('Mol', axis=1)
    df['key'] = 0
    key_cas=key_file[key_file["Drug-Name"]==i.split('.')[0]].iloc[0]['CAS']
    df.loc[df[df["CAS Registry Number"]==key_cas].index,'key'] = 1
    print (i.split('.')[0],len(df[df["key"]==1]))
    df.to_csv (i,index = False,header=True)